In [1]:
# https://www.tensorflow.org/tutorials/images/classification?hl=ko

# 고급 이미지 분류

 이미지에서 고양이 또는 개를 분류 하는 방법을 가지고 실습 !
`tf.keras.Sequential`모델을 사용하여 이미지 분류기를 작성하고를 사용하여 데이터를로드할것이다. 
 
 `tf.keras.preprocessing.image.ImageDataGenerator` 
- 모델과 함께 사용할 디스크의 데이터를 효율적으로 사용하기 위해 클래스를 사용하여 데이터 입력 파이프 라인 구축 tf.keras.preprocessing.image.ImageDataGenerator
- 과적합- 그것을 식별하고 예방하는 방법

- 데이터 확대 및 드롭아웃-컴퓨터 비전 작업에서 과적합하여 데이터 파이프 라인 및 이미지 분류기 모델에 통합하기 위한 주요기술


## 진행 흐름 

1. 데이터 검사 및 이해 
2. 입력 파이프 라인 구출 
3. 모델 구축 
4. 모델 훈련
5. 모델 테스트 
6. 모델 개선 및 프로세스 반복



## 1. 데이터 검사 및 이해

### 패키지 가져오기

os: 파일 및 디렉토리 구조를 판독하는데 사용    
NumPy : 배열 파이썬리스트를 변환하고 요구되는 매트릭스 연산을 수행   
matplotlib.pyplot : 트레이닝 및 검증 데이터의 그래프 표시 이미지를 플롯      

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import tensorflow as tf 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 1.1 데이터 로드 

- 데이터 세트를 다운로드 
    - Kaggle 의 필터링 된 버전의 Dogs vs Cats 데이터 세트를 사용한다.
    - 데이터 세트의 아카이브 버전을 다운로드하여 "/ tmp /"디렉토리에 저장한다. 

In [ ]:
# https://datascienceschool.net/view-notebook/bc7f54b21a8349c79ecaa4906428dea1/

` tf.keras.utils.get_file() ` :  인터넷의 파일을 로컬 컴퓨터의 홈 디렉토리 아래 .keras/datasets 디렉토리로 다운로드

In [5]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

# tf.keras.utils.get_file -> ??? 
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

68608000/68606236 [==============================] - 6s 0us/step


```
cats_and_dogs_filtered 
| __ train 
    | ______ 고양이 : [cat.0.jpg, cat.1.jpg, cat.2.jpg ....]
    | ______ 개 : [dog.0.jpg, dog.1.jpg, dog.2.jpg ...]
| __ validation
    | ______ 고양이 : [cat.2000.jpg, cat.2001.jpg, cat.2002.jpg ....]
    | ______ 개 : [dog.2000.jpg, dog.2001.jpg, dog.2002.jpg ...]
    
```

In [6]:
train_dir = os.path.join(PATH,'train')
validation_dir = os.path.join(PATH,'validation')

In [7]:
# train
train_cats_dir = os.path.join(train_dir, 'cats')  # training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # training dog pictures
# validation
validation_cats_dir = os.path.join(validation_dir, 'cats')  # validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # validation dog pictures

## 1.2 데이터의 이해 

- 훈련 및 검증 디렉토리에 고양이와 개 이미지가 몇개인 지 확인해보자 
- 그리고 편의를 위해서 데이터 세트를 사전 처리하고 네트워크를 훈련하는 동안 사용할 변수를 설정하자

In [8]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

In [10]:
print('총 훈련 고양이 이미지:', num_cats_tr)
print('총 훈련 개 이미지:', num_dogs_tr)

print('총 검증 고양이 이미지:', num_cats_val)
print('총 검증 개 이미지:', num_dogs_val)
print("--")
print("총 학습 이미지:", total_train)
print("총 검증 이미지:", total_val)

총 훈련 고양이 이미지: 1000
총 훈련 개 이미지: 1000
총 검증 고양이 이미지: 500
총 검증 개 이미지: 500
--
총 학습 이미지: 2000
총 검증 이미지: 1000


In [11]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

##  1.3 데이터 준비 

네트워크에 공급하기 전에 이미지를 적절히 사전 처리 된 부동 소수점 텐서로 포멧 

1. 디스크에서 이미지를 읽는다.
2. 이미지 내용을 디코딩해서 RGB 내용에 따라 적절한 격자 형식으로 변환한다. 
3. 부통 소수점 텐서로 변환한다.
4. 신경망은 작은 입력값을 처리하는 것을 선호     
    -> 텐서를 0에서 255 사이의 값에서 0에서 1 사이의 값으로 재조정한다.     
    

모든 작업은 `ImageDataGenerator` 제공하는 클래스로 수행    
tf.keras. 디스크에서 이미지를 읽고 적절한 텐서로 사전 처리 가능하다.     
이러한 이미지를 텐서 배치로 변환하는 생성기를 설정하여 네트워크를 훈련 할 때 유용하다.     

In [12]:
train_image_generator = ImageDataGenerator(rescale=1./255) 
validation_image_generator = ImageDataGenerator(rescale=1./255) 

 훈련 및 유효성 검사 이미지 생성기를 정의한다.       
 flow_from_directory을 이용해 디스크에서 이미지를 로드한다.    
 그리고 크기를 조정한 다음 이미지를 필요한 크기로 조정한다.    

In [13]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 2000 images belonging to 2 classes.


In [14]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 1000 images belonging to 2 classes.


## 교육 이미지 시각화 